# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
!pip install langchain

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install langchain-openai

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [5]:
#!pip install pandas

In [6]:
import pandas as pd
df = pd.read_csv('data/Data.csv')

In [7]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [8]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [9]:
llm = ChatOpenAI(temperature=1)


A lower temperature is between 0 and 0.3 and a higher above 0.5

In order to have some creativity we establish the value at 1, as we have to write product descriptions.

In [10]:
prompt = ChatPromptTemplate.from_template(  
    "Write a funny product description for this product: {product}. "
)

In [11]:
chain = prompt | llm

In [12]:
product = "Plastic nails"
result = chain.invoke(product)
print(result)

content="Introducing the latest trend in nail fashion - plastic nails! Made from the finest plastic materials, these nails provide the perfect solution for when you just can't deal with the hassle of painting and maintaining your nails. Simply attach these bad boys to your natural nails and voila! Instant glam without the mess. Plus, they're completely customizable, so you can switch up your look whenever the mood strikes. Say goodbye to chipped polish and hello to fabulous, no-fuss nails with plastic nails!" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 99, 'prompt_tokens': 20, 'total_tokens': 119, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-161765a7-2a2d-4a6c-af77-7cc1fda9267c-0' usage_metadata={'input_tokens': 20, 'output_to

## SimpleSequentialChain

In [13]:
from langchain.chains import SimpleSequentialChain

In [14]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
"Write a funny product description for this product: {product}. "

)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

/var/folders/qp/3cxfqxvd0dq7rn_xc3tvtcz00000gp/T/ipykernel_16244/2560194775.py:10: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain_one = LLMChain(llm=llm, prompt=first_prompt)


In [15]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Based on the review {review}, rate the quality of this product from zero to ten."
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [16]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [17]:
overall_simple_chain.invoke(product)



> Entering new SimpleSequentialChain chain...
Are you tired of spending hours at the salon getting your nails done, only for them to chip and break within days? Say goodbye to brittle nails and hello to everlasting glamour with our plastic nails! Made from the finest quality plastic, these nails are guaranteed to last longer than your last relationship. Plus, with a variety of colors and styles to choose from, you can easily switch up your look without breaking the bank. So go ahead, treat yourself to a set of plastic nails and get ready to nail that perfect manicure every time!
I would rate this product a 8/10.

> Finished chain.


{'input': 'Plastic nails', 'output': 'I would rate this product a 8/10.'}

In [18]:
for product in df['Product']:
    overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...


/var/folders/qp/3cxfqxvd0dq7rn_xc3tvtcz00000gp/T/ipykernel_16244/1589742392.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  overall_simple_chain.run(product)


Introducing the ultimate royal treatment for your bed - our Queen Size Sheet Set! Made with luxurious materials fit for a queen, these sheets will have you feeling like royalty every time you climb into bed. Say goodbye to tossing and turning on scratchy sheets - our Queen Size Sheet Set will have you sleeping like a monarch in no time. So go ahead, treat yourself to the comfort and style you deserve with our regal sheet set. Your highness, you'll never want to sleep on anything else again!
I would rate the quality of this product a 9 out of 10. The luxurious materials and promise of comfort make it seem like a great investment for a good night's sleep. However, without trying the product myself, I cannot give it a perfect 10 rating.

> Finished chain.


> Entering new SimpleSequentialChain chain...
Introducing the ultimate waterproof phone pouch - because nothing ruins a day at the beach faster than a soggy smartphone! With this handy little pouch, you can text, Snap, and scroll to yo

**Repeat the above twice for different products**

## SequentialChain

In [19]:
from langchain.chains import SequentialChain

In [37]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
  "Translate this review {review} into spanish. "
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key='es_translation',
                      verbose = True
                    )


In [39]:
second_prompt = ChatPromptTemplate.from_template(
    'Based on the review [{es_translation}], summarize it in less than 10 words in Spanish.'
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key='review_summary',
                     verbose = True
                    )


In [40]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    "Translate this review summary [{review_summary}] into german"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key='de_translation',
                       verbose = True
                      )


In [41]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
"""Rate the quality of following translation:
Original text: {review_summary}
German translation: {de_translation}"""
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key='translation_rating',
                      verbose = True
                     )


In [42]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review"],
    output_variables=["translation_rating"],
    verbose=True
)

In [43]:
review = df.Review[3]
print(overall_chain.invoke(review))



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: Translate this review This is the best throw pillow fillers on Amazon. I’ve tried several others, and they’re all cheap and flat no matter how much fluffing you do. Once you toss these in the dryer after you remove them from the vacuum sealed shipping material, they fluff up great into spanish. 

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: Based on the review [Estos son los mejores rellenos de cojín en Amazon. He probado varios otros y todos son baratos y planos sin importar cuánto los esponje. Una vez que los tiras en la secadora después de sacarlos del empaque sellado al vacío, se esponjan muy bien.], summarize it in less than 10 words in Spanish.

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: Translate this review summary [Rellenos de cojín esponjosos y duraderos en Amazon.] into german

> Finishe

**Repeat the above twice for different products or reviews**

## Router Chain

In [26]:
physics_template = """You are a very smart physics professor. 
You are great at answering questions about physics in a concise\
and easy to understand manner. 
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. 
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [44]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

prompt_templates = {
    info["name"]: ChatPromptTemplate.from_template(info["prompt_template"])
    for info in prompt_infos
}

for name, template in prompt_templates.items():
    print(name)
    print(template)
    print()

physics
input_variables=['input'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template="You are a very smart physics professor. You are great at answering questions about physics in a conciseand easy to understand manner. When you don't know the answer to a question you admitthat you don't know.\n\nHere is a question:\n{input}"), additional_kwargs={})]

math
input_variables=['input'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='You are a very good mathematician. You are great at answering math questions. You are so good because you are able to break down hard problems into their component parts, \nanswer the component parts, and then put them togetherto answer the broader question.\n\nHere is a question:\n{input}'), additional_kwargs={})]

History
i

In [28]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [29]:
llm = ChatOpenAI(temperature=0)

In [45]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
print(destinations_str)

physics: Good for answering questions about physics
math: Good for answering math questions
History: Good for answering history questions
computer science: Good for answering computer science questions


In [46]:
default_prompt = ChatPromptTemplate.from_template("Answer this general question as good as you can: {input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [47]:
MULTI_PROMPT_ROUTER_TEMPLATE = """
1. Given a raw text input to a language model select the model prompt best suited for the input.
2. You will be given the names of the available prompts and a description of what the prompt is best suited for.
3. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.
4. REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR  "DEFAULT" if the input is not well suited for any of the candidate prompts.
5. NEVER set as "destination" a prompt name that is not available in the << CANDIDATE PROMPTS >>, use "DEFAULT" instead.
6. REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [51]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

# router_chain = router_prompt | llm | RunnableLambda(lambda x: x.content)
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

# check output, it should be a dict
question = "explain me the theory of relativity"
print(router_chain.invoke(question))

#chain = (
#    RunnablePassthrough()
#    | {
#        "input": RunnablePassthrough(),
#        "chosen_prompt": router_chain,
#    }
#    | RunnableLambda(lambda x: prompt_templates[x["chosen_prompt"]].format(input=x["input"]))
#    | llm
#)
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain,
                         verbose = True)



{'input': 'explain me the theory of relativity', 'destination': 'physics', 'next_inputs': {'input': 'explain the theory of relativity'}}


/var/folders/qp/3cxfqxvd0dq7rn_xc3tvtcz00000gp/T/ipykernel_16244/592422250.py:29: LangChainDeprecationWarning: Use RunnableLambda to select from multiple prompt templates. See example in API reference: https://api.python.langchain.com/en/latest/chains/langchain.chains.router.multi_prompt.MultiPromptChain.html
  chain = MultiPromptChain(router_chain=router_chain,


In [53]:
response = chain.invoke("Explain what principle component analysis is using linear algebra")
print(response)



> Entering new MultiPromptChain chain...
computer science: {'input': 'Explain what principle component analysis is using linear algebra'}
> Finished chain.
{'input': 'Explain what principle component analysis is using linear algebra', 'text': 'Principal component analysis (PCA) is a statistical technique used to simplify data by reducing its dimensionality while retaining as much variation as possible. In essence, PCA finds the directions (or principal components) along which the data points vary the most.\n\nIn linear algebra terms, PCA can be explained as follows:\n\n1. Let X be a matrix containing the data points, where each row represents a different data point and each column represents a different feature. We first center the data by subtracting the mean of each feature from all data points.\n\n2. Next, we compute the covariance matrix of the centered data. The covariance matrix captures the relationships between different features in the data.\n\n3. We then diagonalize the cov

In [54]:
response = chain.invoke("What is black body radiation?")
print(response)



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.
{'input': 'What is black body radiation?', 'text': "Black body radiation refers to the electromagnetic radiation emitted by a perfect absorber and emitter of radiation, known as a black body. A black body absorbs all incoming radiation and emits radiation across the entire spectrum, with the intensity and distribution of the radiation depending on its temperature.\n\nThe spectral distribution of black body radiation follows Planck's law, which describes the intensity of radiation at different wavelengths as a function of temperature. As the temperature of a black body increases, the peak of the radiation spectrum shifts to shorter wavelengths and the overall intensity increases.\n\nBlack body radiation is an important concept in physics and is used to explain various phenomena, such as the color of stars, the thermal radiation emitted by objects, and the behavior of light in

In [55]:
response = chain.invoke("what is 2 + 2")
print(response)



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.
{'input': 'what is 2 + 2', 'text': 'The answer to 2 + 2 is 4.'}


In [57]:
response = chain.invoke("Why does every cell in our body contain DNA?")
print(response)



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}

ValueError: Received invalid destination chain name 'biology'

**Repeat the above at least once for different inputs and chains executions - Be creative!**